In [51]:
import os
import time
from datetime import datetime
import numpy as np
import pandas as pd
from sodapy import Socrata
# import geopandas
# import plotly.express as px
# from shapely.geometry import Polygon, Point
# from d2i_tools import *
import warnings
warnings.simplefilter("ignore")

In [92]:
def dseries(df, col, attrib, attrib_sub=None, count=False):  
    ls = []
    if attrib_sub == None:
        for d in df[col]:
            if count:
                ls.append(len(d[attrib]))
            else:
                ls.append(d[attrib])
    else:
        for d in df[col]:
            if count:
                ls.append(len(d[attrib][attrib_sub]))
            else:
                ls.append(d[attrib][attrib_sub])
    return pd.Series(ls)

def interesteddf(rdf):
    a = dseries(rdf, 'resource', 'name').str.lower()
    b = dseries(rdf, 'resource', 'id')
    c = dseries(rdf, 'resource', 'parent_fxf')
    d = rdf.permalink
    e = dseries(rdf, 'resource', 'description').str.lower()
    f = dseries(rdf, 'resource', 'data_updated_at')
    f = pd.to_datetime(f, infer_datetime_format=True)
    g = dseries(rdf, 'resource', 'page_views', 'page_views_last_week')
    h = dseries(rdf, 'resource', 'page_views', 'page_views_last_month')
    i = dseries(rdf, 'resource', 'page_views', 'page_views_total')
    j = dseries(rdf, 'resource', 'download_count')
    k = dseries(rdf, 'classification', 'categories')
    l = dseries(rdf, 'classification', 'domain_category').str.lower()
    m = dseries(rdf, 'classification', 'domain_tags')
    n = dseries(rdf, 'classification', 'domain_metadata')
    col =  ['name', 'id', 'parent_fxf', 'permalink', 'description', 'data_upd_at', 'pv_last_wk', 
            'pv_last_mth', 'pv_total', 'download_count', 'categories', 'domain_category', 
            'domain_tags', 'domain_metadata']
    df = pd.concat([a,b,c,d,e,f,g,h,i,j,k,l,m,n], axis='columns')
    df.columns = col
    return df

In [100]:
def getMeta(apptoken=None):
    client = Socrata("data.melbourne.vic.gov.au", apptoken)
    rdf = pd.DataFrame.from_dict(client.datasets())
    ds_df = interesteddf(rdf)
    df = ds_df.copy()

    keys = []
    for record in ds_df['domain_metadata']:
            for kv in record:
                keys.append(kv['key'])
    keys_ds = pd.Series(keys).drop_duplicates().reset_index(drop=True)

    for key in keys_ds:
        df[key] = np.nan
        for i in range(df.shape[0]):
            for kv in df['domain_metadata'][i]:
                if kv['key'] == key:
                    df[key][i] = kv['value'].lower()

    # disinterested domain_metadata
    dis = ['How-to-use_Applicable-standard-(URL)',\
           'Internal-management_Source-system-(GIS,-AssetMaster,-etc)',
           'Internal-management_Update-mechanism',
           'Internal-management_IMGF-risk-level',
           'Melbourne-Metadata_Further-Information',
           'Quality_Source-data-update-frequency',
           'domain_metadata']
    return df.drop(columns=dis)    

In [102]:
getMeta("wMEkdLbVuXIpLiCFVic1PgiZ3")

,name,id,parent_fxf,permalink,description,data_upd_at,pv_last_wk,pv_last_mth,pv_total,download_count,...,domain_category,domain_tags,Quality_What's-included,Quality_Update-frequency,Quality_Reliability-level,How-to-use_Linked-to,Data-management_Source-data-update-frequency,Quality_Known-Issues,How-to-use_Further-information,Quality_Data-quality-statement
0,pedestrian counting system - monthly (counts p...,b2ak-trbp,[],https://data.melbourne.vic.gov.au/d/b2ak-trbp,this dataset contains hourly pedestrian counts...,2021-11-04 22:38:23+00:00,126,829,73821,8831,...,transport,"[accessibility, cityreactivation, covid-19, fo...",all data held by the city of melbourne has bee...,monthly,reliable and timely,http://www.pedestrian.melbourne.vic.gov.au/,hourly,NaN,NaN,NaN
1,tree canopies 2011 (urban forest),y79a-us3f,[],https://data.melbourne.vic.gov.au/d/y79a-us3f,tree canopy within city of melbourne mapped us...,NaT,5,36,66898,3178,...,environment,"[citizenreporter, eventsplanner, forest, tree,...",NaN,static (not updated),useful with caveats,NaN,monthly,NaN,NaN,NaN
2,"trees, with species and dimensions (urban forest)",fp38-wiyy,[],https://data.melbourne.vic.gov.au/d/fp38-wiyy,"the city of melbourne maintains more than 70,0...",2021-09-30 16:06:54+00:00,49,302,39828,6393,...,environment,"[accessibility, changemaker, citizenreporter, ...",all data held by the city of melbourne has bee...,daily,reliable and timely,NaN,monthly,date and year planted fields may have missing ...,NaN,NaN
3,on-street parking bay sensors,vh2v-4nfs,[],https://data.melbourne.vic.gov.au/d/vh2v-4nfs,contains information from in-ground car parkin...,2021-11-17 07:46:28+00:00,162,656,38565,12084336,...,transport,"[parking, sensor, near real-time, vacancy, tra...",all active sensor readings. number of sensor r...,2 minutes,useful with caveats,NaN,real-time,parking sensors are not operational on public ...,http://www.melbourne.vic.gov.au/parking-and-tr...,NaN
4,pay stay zones linked to street segments,7q9g-yyvg,[],https://data.melbourne.vic.gov.au/d/7q9g-yyvg,this dataset contains each pay stay zone and t...,2018-03-02 01:46:14+00:00,114,467,32049,1814,...,transport,"[parking, paystay, street, travel]",NaN,static (not updated),useful with caveats,NaN,monthly,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,cardboard bales collected at degraves st recyc...,hisn-tyup,[],https://data.melbourne.vic.gov.au/d/hisn-tyup,this dataset records the number of bales (bund...,2017-05-12 06:23:18+00:00,0,3,225,844,...,environment,"[bales, cardboard, dsrf, recycling, waste]",NaN,static (not updated),useful with caveats,NaN,daily,NaN,NaN,NaN
217,city of melbourne population forecasts by smal...,sp4r-xphj,[],https://data.melbourne.vic.gov.au/d/sp4r-xphj,this dataset provides population forecasts by ...,2021-08-12 06:45:16+00:00,12,83,109,17,...,people,"[age, demographics, forecast, gender, growth, ...",the latest city of melbourne population foreca...,annually,reliable and timely,NaN,annually,NaN,NaN,NaN
218,city of melbourne jobs forecasts by small area...,73qa-862u,[],https://data.melbourne.vic.gov.au/d/73qa-862u,this dataset provides jobs forecasts by single...,2021-08-12 07:19:52+00:00,11,39,51,13,...,people,"[business, employment, forecast, growth, indus...",NaN,annually,reliable and timely,NaN,annually,NaN,NaN,NaN
219,city of melbourne dwellings and household fore...,jizx-9ayp,[],https://data.melbourne.vic.gov.au/d/jizx-9ayp,this dataset provides dwellings and households...,2021-08-12 07:01:29+00:00,2,26,47,7,...,people,"[dwellings, forecast, growth, households, smal...",the latest city of melbourne dwellings and hou...,annually,reliable and timely,NaN,annually,NaN,NaN,NaN


In [99]:
client = Socrata("data.melbourne.vic.gov.au", None)

rdf = pd.DataFrame.from_dict(client.datasets())
ds_df = interesteddf(rdf)
df1 = ds_df.copy()

keys = []
for record in ds_df['domain_metadata']:
        for kv in record:
            keys.append(kv['key'])
keys_ds = pd.Series(keys).drop_duplicates().reset_index(drop=True)

for key in keys_ds:
    df1[key] = np.nan
    for i in range(df1.shape[0]):
        for kv in df1['domain_metadata'][i]:
            if kv['key'] == key:
                df1[key][i] = kv['value'].lower()

# disinterested domain_metadata
dis = ['How-to-use_Applicable-standard-(URL)',\
       'Internal-management_Source-system-(GIS,-AssetMaster,-etc)',
       'Internal-management_Update-mechanism',
       'Internal-management_IMGF-risk-level',
       'Melbourne-Metadata_Further-Information',
       'Quality_Source-data-update-frequency',
       'domain_metadata']
df1 = df1.drop(columns=dis)
print(f'The shape of extracted meta data dataframe is {df1.shape}.')
print('Below are my selected meta data for parking and pedestrian snapshot datasets:')
df1[df1.id.isin(['vh2v-4nfs', 'd6mv-s43h'])].T

The shape of extracted meta data dataframe is (221, 21).
Below are my selected meta data for parking and pedestrian snapshot datasets:


,3,28
name,on-street parking bay sensors,pedestrian counting system - past hour (counts...
id,vh2v-4nfs,d6mv-s43h
parent_fxf,[],[]
permalink,https://data.melbourne.vic.gov.au/d/vh2v-4nfs,https://data.melbourne.vic.gov.au/d/d6mv-s43h
description,contains information from in-ground car parkin...,<b>current issue 23/09/2020</b>\nplease note: ...
data_upd_at,2021-11-17 07:42:29+00:00,2021-11-17 07:31:57+00:00
pv_last_wk,162,36
pv_last_mth,656,127
pv_total,38565,7930
download_count,12084336,20699
